### Idea Working Towards:
- Load well with LASIO
- Export LASIO style JSON
- CONVERT LASIO style JSON to WELLIO style JSON
- Visualize with wellioviz

#### Note !! 
Node variables don't seem to be re-initializable once they are defined via pixiedust_node.

To work-around this:
when needing to reset a node variable, restart the Jupyter Kernal.

In [ ]:
import json
import lasio
import numpy as np
from datetime import datetime

#### Pixiedust node module enables running Node.js code in Jupyter.
https://github.com/pixiedust/pixiedust_node

In [ ]:
import pixiedust_node

#### Install wellio and wellioviz if not already installed.

In [ ]:
%%node
var install_wellio = 0;
try {
    require.resolve('wellio');    
} catch (e) {
    if (e.code === 'MODULE_NOT_FOUND') {
        console.log("Will install Wellio.js")
        install_wellio = 1;
    } else {
        console.log(e);
        install_wellio = 1;
    }
}

In [ ]:
if install_wellio:
    npm.install('wellio')
    require.resolve('wellio')

In [ ]:
%%node
var install_wellioviz = 0;
try {
    require.resolve('wellioviz');    
} catch (e) {
    if (e.code === 'MODULE_NOT_FOUND') {
        console.log("Will install Wellioviz.js");
        install_wellioviz = 1;
    } else {
        console.log(e);
        install_wellioviz = 1;
    }
}

In [ ]:
if install_wellioviz:
    npm.install('wellioviz')

In [ ]:
%%node
var wellio = null;
var wellioviz = null;

try {
    wellio = require('wellio');
    wellioviz = require('wellioviz');
} catch (err) {
    console.log(err);
}

#### Create a las file via lasio and translate it to json.

In [ ]:
las = lasio.LASFile()

las.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
las.params['ENG'] = lasio.HeaderItem('ENG', value='Kent Inverarity')
las.params['LMF'] = lasio.HeaderItem('LMF', value='GL')
las.other = 'Example of how to create a LAS file from scratch using lasio'
depths = np.arange(10, 50, 0.5)
synth = np.log10(depths)*5+np.random.random(len(depths))
synth[:8] = np.nan
las.add_curve('DEPT', depths, unit='m')
las.add_curve('SYNTH', synth, descr='fake data')
las.write('scratch_v2.las', version=2)
json_images = json.dumps(las, cls=lasio.JSONEncoder)

#### Example 1: Read the in-memory Lasio json string into Wellio json

In [ ]:
%%node
var wellio_obj = '';

try {
    let lasio_obj = '';
    lasio_obj = JSON.parse(json_images);
    wellio_obj = wellio.lasio_obj_2_wellio_obj(lasio_obj);
} catch (e) {
    console.log('[');
    console.log(e.name + ":: " + e.message);
    console.log(']');
}

console.log(wellio_obj);



#### Introduce wellioviz and pass the wellio_obj to wellioviz

In [ ]:
%%node
console.log(wellioviz.define_wellioviz());

In [ ]:
%%node
let three_things = wellioviz.fromJSONofWEllGetThingsForPlotting(wellio_obj,"DEPT");
console.log(three_things);

#### Example 2: Write Lasio json to file, then get current path and read the created json file into wellio json

In [ ]:
las_json_dict =json.loads(json_images)
with open('data.json', 'w') as outfile:
    json.dump(las_json_dict, outfile)

In [ ]:
%%node
const path = require('path');
let mydir = process.env.PWD;
let myfile = mydir + path.sep + 'data.json';

let lasio_json_str = '';
let lasio_obj_2 = '';
let wellio_obj_2 = '';

try {
    lasio_json_str = wellio.read_lasio_json_file(myfile);
    lasio_obj_2 = JSON.parse(lasio_json_str);
    wellio_obj_2 = wellio.lasio_obj_2_wellio_obj(lasio_obj_2);
} catch (e) {
    console.log('[');
    console.log(e.name + ":: " + e.message);
    console.log(']');
}

console.log(wellio_obj_2);


#### Example 3: Read created Lasio Las file

In [ ]:
%%node
three_things_2 = wellioviz.fromJSONofWEllGetThingsForPlotting(wellio_obj,"DEPT");
console.log(three_things_2);

In [ ]:
%%node
const path = require('path');
let mydir_3 = process.env.PWD;
let myfile_3 = mydir + path.sep + 'scratch_v2.las';

let las_str_3 = '';
let wellio_obj_3 = '';

try {
    las_str_3 = wellio.loadLAS(myfile_3);

    wellio_obj_3 = wellio.las2json(las_str_3);
} catch (e) {
    console.log('[');
    console.log(e.name + ":: " + e.message);
    console.log(']');
}
console.log(wellio_obj_3);



In [ ]:
%%node
three_things_3 = wellioviz.fromJSONofWEllGetThingsForPlotting(wellio_obj,"DEPT");
console.log(three_things_3);